# Bài toán cá hồi

*Mô hình hóa và mô phỏng bằng Python*

Bản quyền 2021 Allen Downey

Giấy phép: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# cài đặt Pint nếu cần

try:
    import pint
except ImportError:
    !pip install pint

In [2]:
# tải về modsim.py nếu cần

from os.path import exists

filename = 'modsim.py'
if not exists(filename):
    from urllib.request import urlretrieve
    url = 'https://raw.githubusercontent.com/AllenDowney/ModSim/main/'
    local, _ = urlretrieve(url+filename, filename)
    print('Downloaded ' + local)

In [3]:
# import functions from modsim

from modsim import *

## Can we predict salmon populations?

Hàng năm, [Ủy ban đánh giá cá hồi Đại Tây Dương của Hoa Kỳ](https://www.nefsc.noaa.gov/USASAC/Reports/USASAC2018-Report-30-2017-Activities.pdf) đều báo cáo con số ước tính số cá hồi trên biển và trong sông ở Đông bắc Hoa Kỳ. Những báo cáo này rất hữu ích để giám sát những thay đổi về số lượng cá, nhưng nói chung không bao gồm các con số dự đoán.

Mục tiêu của nghiên cứu cụ thể này nhằm mô phỏng thay đổi hằng năm của số cá hồi, đánh giá xem những thay đổi này có dễ dự đoán không, và ước tính xác suất để một số cá cụ thể sẽ tăng hay giảm trong vòng 10 năm tới.

Lấy ví dụ, tôi dùng dữ liệu từ trang 18 của cuốn báo cáo năm 2017, vốn cung cấp những số cá ước tính cho các dòng sông Narraguagus và Sheepscot ở bang Maine.

![USASAC_Report_2017_Page18](https://github.com/AllenDowney/ModSim/raw/main/data/USASAC_Report_2017_Page18.png)

Có những công cụ để tự động kết xuất dữ liệu từ một tài liệu PDF, nhưng ở ví dụ này tôi sẽ đơn giản là tự gõ mã lệnh.

Đây là những ước tính số cá hồi cho dòng sông Narraguagus:

In [4]:
pops = [2749, 2845, 4247, 1843, 2562, 1774, 1201, 1284, 1287, 
        2339, 1177, 962, 1176, 2149, 1404, 969, 1237, 1615, 1201]

Để lấy được dữ liệu này vào một dãy Series trong Pandas, tôi cũng sẽ lập một dãy các năm để dùng làm chỉ số.

In [10]:
years = linrange(1997, 2015)
years

Và đây là dãy số liệu.

In [24]:
pop_series = TimeSeries(pops, index=years)
pop_series

Đây là những gì kết quả:

In [14]:
def plot_population(series):
    series.plot(label='Số cá ước tính')
    decorate(xlabel='Năm', 
             ylabel='Số cá hồi ước tính', 
             title='Sông Narraguacus',
             ylim=[0, 5000])
    
plot_population(pop_series)

## Mô hình hóa những thay đổi

Để thấy số cá hồi thay đổi thế nào qua các năm, tôi sẽ dùng `diff` để tính trị tuyệt đối hiệu số giữa hai năm kế tiếp.


In [25]:
abs_diffs = pop_series.diff()
abs_diffs

Ta có thể tính hiệu số tương đối bằng cách đem chia dãy số gốc cho từng phần tử.

In [26]:
rel_diffs = abs_diffs / pop_series
rel_diffs

Những hiệu số tương đối này là tốc độ tăng trưởng ròng quan sát được hàng năm. Bởi vậy ta hãy xóa số `0` rồi lưu chúng lại.

In [27]:
rates = rel_diffs.dropna()
rates

Một cách đơn giản để mô hình hóa hệ thống này là rút một giá trị ngẫu nhiên từ dãy số mức tăng trưởng hằng năm quan sát được này. Ta có thể dùng hàm `choice` của NumPy để lựa chọn ngẫu nhiên từ một dãy số.

In [28]:
np.random.choice(rates)

## Mô phỏng

Bây giờ ta có thể mô phỏng hệ thống bằng cách rút ra những trị số tăng trưởng ngẫu nhien từ các trị số quan sát được.

Tôi sẽ bắt đầu mô phỏng cho năm 2015.

In [29]:
t_0 = 2015
p_0 = pop_series[t_0]

Tôi sẽ tạo một đối tượng `System` với các biến `t_0`, `p_0`, `rates`, và `duration=10` năm. 

Dãy số tăng trưởng quan sát thấy chính là tham số quan trọng của mô hình.

In [30]:
system = System(t_0=t_0,
                p_0=p_0,
                duration=10,
                rates=rates)

Hãy viết một hàm cập nhật để nhận vào các tham số `pop`, `t`, và `system`.
Nó phải chọn được một suất tăng trưởng ngẫu nhiên, tính ra mức thay đổi số cá hồi, và trả lại số cá mới.

In [31]:
# Lời giải

Kiểm thử hàm cập nhật mà bạn viết và chạy nó vài lần

In [32]:
update_func1(p_0, t_0, system)

Đây là một phiên bản `run_simulation` có lưu kết quả trong một `TimeSeries` rồi trả lại nó.

In [33]:
def run_simulation(system, update_func):
    """Mô phỏng một hệ thống hàng đợi.
    
    system: đối tượng System
    update_func: đối tượng hàm
    """
    t_0 = system.t_0
    t_end = t_0 + system.duration
    
    results = TimeSeries()
    results[t_0] = system.p_0
    
    for t in linrange(t_0, t_end):
        results[t+1] = update_func(results[t], t, system)

    return results

Dùng `run_simulation` để chạy phát sinh một con số dự tính cho 10 năm tới.

Biểu đồ thể hiện dự đoán so với số liệu gốc. Dự đoán của bạn bắt đầu từ chỗ mà dữ liệu kết thúc.

In [35]:
# Lời giải

Để hình dung được kết quả thay đổi ở mức độ nào, ta có thể chạy mô hình vài lần và vẽ đồ thị tất cả kết quả.

In [42]:
def plot_many_simulations(system, update_func, iters):
    """Chạy mô phỏng và vẽ đồ thị kết quả.
    
    system: đối tượng System
    update_func: đối tượng hàm
    iters: số lần chạy mô phỏng
    """
    for i in range(iters):
        results = run_simulation(system, update_func)
        results.plot(color='gray', label='_nolegend', 
                     linewidth=1, alpha=0.3)

Tùy chọn đồ thị `alpha=0.1` khiến cho các đường nét một phần trong suốt, nhờ vậy khi chồng lên nhau nét vẽ sẽ đậm hơn.

Chạy `plot_many_simulations` với hàm cập nhật bạn viết và `iters=30`.  Cũng vẽ luôn số liệu gốc.

In [43]:
# Lời giải

Các kết quả biến động mạnh mẽ: theo mô hình này, số cá hồi có thể tiếp tục giảm trong 10 năm tới, hoặc có thể hồi phục và tăng nhanh!

Thật khó nói ta có thể tin tưởng mô hinh này đến đâu. Có nhiều yếu tố ảnh hưởng đến số cá hồi mà không được đề cập tới trong mô hình. Chẳng hạn, nếu số cá bắt đâu tăng nhanh, nó có thể bị hạn chế bởi các giới hạn tài nguyên, các loài ăn thịt, hay sự đánh bắt cá. Nếu số cá bắt đầu giảm, người ta có thể hạn chế đánh bắt cá và dồn các con cá được nuôi vào cho sông.

Bởi vậy những kết quả này có thể chưa được coi rằng những dự đoán hữu ích. Tuy nhiên, còn có điều hữu ích mà ta có thể làm, đó là ước tính xác suất để số cá hồi sẽ tăng hoặc giảm trong 10 năm tới.  

## Phân bố của mức biến đổi tịnh

Để mô tả sự phân bố của những biến đổi tịnh, hãy viết một hàm có tên `run_many_simulations` để chạy nhiều mô phỏng, lưu lại số cá hồi cuối cùng vào một `ModSimSeries`, rồi trả lại `ModSimSeries` đó.


In [45]:
def run_many_simulations(system, update_func, iters):
    """Chạy mô phỏng và báo lại số cá cuối cùng.
    
    system: đối tượng System
    update_func: đối tượng hàm
    iters: số lần chạy mô phỏng
    
    trả lại: một dãy các số cá cuối cùng
    """
    # ĐIỀN MÃ LỆNH VÀO

In [50]:
# Lời giải

Kiểm thử hàm bạn viết bằng cách chạy nó với `iters=5`.

In [51]:
run_many_simulations(system, update_func1, 5)

Bây giờ ta có thể chạy 1000 mô phỏng và mô tả phân bố kết quả.

In [52]:
last_pops = run_many_simulations(system, update_func1, 1000)
last_pops.describe()

Nếu ta trừ đi số cá ban đầu, ta được phân bố các sự thay đổi.

In [53]:
net_changes = last_pops - p_0
net_changes.describe()

Số trung vị là một số âm, vốn cho thấy rằng dân số bị giảm thường xuyên hơn là tăng.

Ta có thể cụ thể hơn bằng cách đếm số lần liên tiếp mà `net_changes` là dương.

In [54]:
np.sum(net_changes > 0)

Hoặc ta có thể dùng `mean` để tính tỉ lệ những lần liên tiếp mà `net_changes` là số dương.

In [55]:
np.mean(net_changes > 0)

Và đây là tỉ lệ số lần liên tiếp âm.

In [56]:
np.mean(net_changes < 0)

Vì vậy, dựa trên những thay đổi trong quá khứ quan sát được, mô hình này dự báo rằng số cá có nhiều khả năng giảm hơn là tăng trong 10 năm tới, với tỉ lệ khoảng 2:1.

## Mô hình tinh chỉnh

Có một số cách mà ta có thể cải thiện mô hình này.

1.  Có vẻ như dữ liệu quá khứ có biểu hiện tuần hoàn, với chu kì khoảng 4-5 năm. Ta có thể mở rộng mô hình để bao gồm hiệu ứng này.

2.  Các dữ liệu khác, nếu so với các dữ liệu mới, có thể không liên quan bằng cho việc dự đoán; vì vậy ta sẽ đặt trọng số lớn hơn vào dữ liệu mới.

Lựa chọn thứ hai dễ thực hiện hơn, vì vậy ta hãy cùng thử nó.

Tôi sẽ dùng `linspace` để tạo một mảng các trọng số "weights" cho các độ tăng quan sát được. Xác suất mà ta chọn từng độ tăng sẽ tỉ lệ với các trọng số này.

Các trọng số phải cộng lại bằng 1, vì vậy tôi chia từng độ tăng cho tổng số này.

In [66]:
weights = linspace(0, 1, len(rates))
weights /= sum(weights)
weights

Tôi sẽ cộng các trọng số vào đối tượng `System`, vì chúng là các tham số của mô hình.

In [59]:
system.weights = weights

Ta có thể truyền các trọng số này như là tham số cho `np.random.choice` (xem [tài liệu](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html))

In [60]:
np.random.choice(system.rates, p=system.weights)

Hãy viết một hàm cập nhật có tính đến các trọng số.

In [61]:
# Lời giải

Dùng `plot_many_simulations` để vẽ đồ thị kết quả.

In [62]:
# Lời giải

Hãy dùng `run_many_simulations` để thu thập kết quả và `describe` để tóm tắt sự phân bố của các mức thay đổi.

In [63]:
# Lời giải

Liệu mô hình tinh chỉnh có nhiêu ảnh hưởng đến xác suất mà số cá hồi giảm đi?

In [64]:
# Lời giải